In [12]:
# Impport necessary libraries
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI


In [13]:
# Connect your MySQL database
# Make sure to install the required packages
host = 'localhost'
port = '3306'
username = 'root'
password = 'root'
database_schema = 'regional_sales_data'
mysql_uri = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_schema}"
db = SQLDatabase.from_uri(mysql_uri, sample_rows_in_table_info=2)

In [14]:
# Database connection
db = SQLDatabase.from_uri(mysql_uri, sample_rows_in_table_info=1)

In [15]:
context =db.get_table_info()

In [16]:
context

'\nCREATE TABLE `2017_budgets` (\n\t`Product Name` TEXT, \n\t`2017 Budgets` DOUBLE\n)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci\n\n/*\n1 rows from 2017_budgets table:\nProduct Name\t2017 Budgets\nProduct 1\t3016489.2089999998\n*/\n\n\nCREATE TABLE customers (\n\t`Customer Index` INTEGER, \n\t`Customer Names` TEXT\n)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci\n\n/*\n1 rows from customers table:\nCustomer Index\tCustomer Names\n1\tGeiss Company\n*/\n\n\nCREATE TABLE products (\n\t`Index` INTEGER, \n\t`Product Name` TEXT\n)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci\n\n/*\n1 rows from products table:\nIndex\tProduct Name\n1\tProduct 1\n*/\n\n\nCREATE TABLE regions (\n\tid INTEGER, \n\tname TEXT, \n\tcounty TEXT, \n\tstate_code TEXT, \n\tstate TEXT, \n\ttype TEXT, \n\tlatitude DOUBLE, \n\tlongitude DOUBLE, \n\tarea_code INTEGER, \n\tpopulation INTEGER, \n\thouseholds INTEGER, \n\tmedian_income INTEGER, \n\tland_area INTEGER, \

In [17]:
# Create the LLM Prompt Template                  
from langchain_core.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a SQL query that would answer the user's question:
Remember : Only provide me the sql query dont include anything else.
           Provide me sql query in a single line dont add line breaks.
Table Schema:
{schema}

Question: {question}
SQL Query:
"""
prompt = ChatPromptTemplate.from_template(template)

In [18]:
# get the schema of the database
def get_schema(db):
    schema = db.get_table_info()
    return schema

In [19]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    api_key="API_KEY_HERE"
)

In [20]:
# Create the SQL query chain using the LLM and the prompt template
sql_chain = (
    RunnablePassthrough.assign(schema=lambda _: get_schema(db))
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [21]:
#test the SQL query chain with a sample question
resp=sql_chain.invoke({"question": "What was the budget of Product 12"})
print(resp)

```sql
SELECT `2017 Budgets` FROM `2017_budgets` WHERE `Product Name` = 'Product 12'
```


In [5]:
import re

query = re.search(r"```sql\s*(.*?)\s*```", resp, re.DOTALL | re.IGNORECASE)

if query:
    query=query.group(1).strip()

In [7]:
query

"SELECT `2017 Budgets` FROM `2017_budgets` WHERE `Product Name` = 'Product 12'"

In [23]:
db.run(query)

'[(1356976.996,)]'

### RAGAS Implementation

In [24]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

In [25]:
#from langchain_openai import ChatOpenAI
#from langchain.embeddings import OpenAIEmbeddings

In [26]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="gemma2-9b-it",
    api_key="API_KEY_HERE"
 
)



In [27]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [28]:
evaluator_llm = LangchainLLMWrapper(llm)
evaluator_embeddings = LangchainEmbeddingsWrapper(embeddings)

In [29]:
from ragas import evaluate
from ragas.metrics import AspectCritic, RubricsScore

maliciousness_definition = (
    "Is the response intended to harm, deceive, or exploit users?"
)

aspect_critic = AspectCritic(
    name="maliciousness",
    definition=maliciousness_definition,
    llm=evaluator_llm,
)

# adapeted google's helpfulness_prompt_template
helpfulness_rubrics = {
    "score1_description": "Response is useless/irrelevant, contains inaccurate/deceptive/misleading information, and/or contains harmful/offensive content. The user would feel not at all satisfied with the content in the response.",
    "score2_description": "Response is minimally relevant to the instruction and may provide some vaguely useful information, but it lacks clarity and detail. It might contain minor inaccuracies. The user would feel only slightly satisfied with the content in the response.",
    "score3_description": "Response is relevant to the instruction and provides some useful content, but could be more relevant, well-defined, comprehensive, and/or detailed. The user would feel somewhat satisfied with the content in the response.",
    "score4_description": "Response is very relevant to the instruction, providing clearly defined information that addresses the instruction's core needs.  It may include additional insights that go slightly beyond the immediate instruction.  The user would feel quite satisfied with the content in the response.",
    "score5_description": "Response is useful and very comprehensive with well-defined key details to address the needs in the instruction and usually beyond what explicitly asked. The user would feel very satisfied with the content in the response.",
}

rubrics_score = RubricsScore(name="helpfulness", rubrics=helpfulness_rubrics, llm=evaluator_llm)

In [30]:
from ragas import evaluate
from ragas.metrics import ContextPrecision, Faithfulness

context_precision = ContextPrecision(llm=evaluator_llm)
faithfulness = Faithfulness(llm=evaluator_llm)

In [32]:
retrieved_contexts = [context]

In [45]:
import re

user_inputs = [
    "What was the budget of Product 12",
    "What are the names of all products in the products table?",
    "List all customer names from the customers table.",
    "Find the name and state of all regions in the regions table.",
    "What is the name of the customer with Customer Index = 1"
]

responses = []

for question in user_inputs:
    resp = sql_chain.invoke({"question": question})
    match = re.search(r"```sql\s*(.*?)\s*```", resp, re.DOTALL | re.IGNORECASE)
    if match:
        query = match.group(1).strip()
        responses.append(query)

In [46]:
references=["SELECT `2017 Budgets` FROM `2017_budgets` WHERE `Product Name` = 'Product 12';",
            "SELECT `Product Name`ROM products;",
            "SELECT `Customer Names`FROM customers;",
            "SELECT name, state FROM regions;",
            "SELECT `Customer Names` FROM customers WHERE `Customer Index` = 1;"]

In [47]:
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset


In [48]:
n = len(user_inputs)
samples = []

In [49]:
for i in range(n):

    sample = SingleTurnSample(
        user_input=user_inputs[i],
        retrieved_contexts=list(retrieved_contexts),
        response=responses[i],
        reference=references[i],
    )
    samples.append(sample)

In [50]:
ragas_eval_dataset = EvaluationDataset(samples=samples)
ragas_eval_dataset.to_pandas()

,user_input,retrieved_contexts,response,reference
0,What was the budget of Product 12,[\nCREATE TABLE `2017_budgets` (\n\t`Product N...,SELECT `2017 Budgets` FROM `2017_budgets` WHER...,SELECT `2017 Budgets` FROM `2017_budgets` WHER...
1,What are the names of all products in the prod...,[\nCREATE TABLE `2017_budgets` (\n\t`Product N...,SELECT `Product Name` FROM products,SELECT `Product Name`ROM products;
2,List all customer names from the customers table.,[\nCREATE TABLE `2017_budgets` (\n\t`Product N...,SELECT `Customer Names` FROM customers,SELECT `Customer Names`FROM customers;
3,Find the name and state of all regions in the ...,[\nCREATE TABLE `2017_budgets` (\n\t`Product N...,"SELECT name, state FROM regions;","SELECT name, state FROM regions;"
4,What is the name of the customer with Customer...,[\nCREATE TABLE `2017_budgets` (\n\t`Product N...,SELECT `Customer Names` FROM customers WHERE `...,SELECT `Customer Names` FROM customers WHERE `...


In [53]:
from ragas import evaluate

ragas_metrics = [ context_precision, rubrics_score]

result = evaluate(
    metrics=ragas_metrics,
    dataset=ragas_eval_dataset
)
result

Evaluating: 100%|██████████| 10/10 [00:01<00:00,  7.27it/s]


{'context_precision': 1.0000, 'helpfulness': 3.8000}